In [4]:
import websockets
import json
from datetime import datetime, timezone
import sys
import os
import asyncio

# Add project root to path for importing config
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Import settings from config.py (dotenv is already loaded there)
from config import AIS_API_KEY, AIS_STREAM_URL, AIS_BOUNDING_BOXES, DB_CONFIG


In [6]:
async def connect_ais_stream():
    # Dictionary to store ship names (ship_id -> name)
    ship_names = {}
    
    async with websockets.connect(AIS_STREAM_URL) as websocket:
        subscribe_message = {
            "APIKey": AIS_API_KEY, 
            "BoundingBoxes": AIS_BOUNDING_BOXES
        }
        
        subscribe_message_json = json.dumps(subscribe_message)
        await websocket.send(subscribe_message_json)
        
        message_count = 0
        
        async for message_json in websocket:
            message = json.loads(message_json)
            message_type = message["MessageType"]
            
            # Handle ShipStaticData to get ship names
            if message_type == "ShipStaticData":
                static_data = message.get('Message', {}).get('ShipStaticData', {})
                ship_id = static_data.get('UserID')
                ship_name = static_data.get('Name', '').strip()
                if ship_id and ship_name:
                    # Check if this is a new name or update
                    was_unknown = ship_id not in ship_names
                    ship_names[ship_id] = ship_name
                    if was_unknown:
                        # Log when we get a new ship name
                        print(f"Got name for ShipID {ship_id}: {ship_name}")
            
            if message_type == "PositionReport":
                # the message parameter contains a key of the message type which contains the message itself
                ais_message = message['Message']['PositionReport']
                message_count += 1
                
                # Extract data
                ship_id = ais_message.get('UserID', 'N/A')
                lat = ais_message.get('Latitude', 0)
                lon = ais_message.get('Longitude', 0)
                speed = ais_message.get('SpeedOverGround', None)
                course = ais_message.get('CourseOverGround', None)
                heading = ais_message.get('Heading', None)
                
                # Get ship name from cache
                ship_name = ship_names.get(ship_id, 'Unknown')
                
                # Format time
                time_str = datetime.now(timezone.utc).strftime("%H:%M:%S")
                
                # Build output string
                info_parts = []
                if speed is not None:
                    info_parts.append(f"Speed: {speed:.1f} kn")
                if course is not None:
                    info_parts.append(f"Course: {course:.1f}°")
                if heading is not None:
                    info_parts.append(f"Heading: {heading:.1f}°")
                
                info_str = " | ".join(info_parts) if info_parts else ""
                
                # Print formatted output with ship name
                print(f"[{time_str}] #{message_count:4d} | ShipID: {ship_id:12d} | "
                      f"Name: {ship_name:20s} | "
                      f"Lat: {lat:8.5f}° | Lon: {lon:9.5f}°" + 
                      (f" | {info_str}" if info_str else ""))

await connect_ais_stream()

[16:06:46] #   1 | ShipID:    477996332 | Name: Unknown              | Lat: 22.30428° | Lon: 114.15636°
[16:06:46] #   2 | ShipID:    314871000 | Name: Unknown              | Lat:  1.27780° | Lon: 103.91881°
[16:06:46] #   3 | ShipID:    525022391 | Name: Unknown              | Lat: -7.19692° | Lon: 112.72982°
[16:06:46] #   4 | ShipID:    563079960 | Name: Unknown              | Lat:  1.25667° | Lon: 103.82915°
[16:06:47] #   5 | ShipID:    219692000 | Name: Unknown              | Lat:  1.19737° | Lon: 103.52722°
[16:06:47] #   6 | ShipID:    563032960 | Name: Unknown              | Lat:  1.20344° | Lon: 103.84643°
[16:06:47] #   7 | ShipID:    636093403 | Name: Unknown              | Lat:  1.23619° | Lon: 103.80599°
[16:06:47] #   8 | ShipID:    413490310 | Name: Unknown              | Lat: 22.30855° | Lon: 114.15431°
📝 Got name for ShipID 563180600: MARINE DYNAMO
📝 Got name for ShipID 370734000: ATA-M
[16:06:47] #   9 | ShipID:    574222000 | Name: Unknown              | Lat:  1.275

CancelledError: 